### Initilize the SparkSession

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

In [7]:
spark

### Reading data using spark

In [11]:
sample = spark.read.csv('sample_data.csv', header=True)

In [12]:
type(sample)

pyspark.sql.dataframe.DataFrame

In [13]:
sample.show()

+-------+-----------+--------------------+--------------------+---------+--------+-------------------+-------+--------+----+-------------------+
|stud_id|actual_abil|        testing_abil|        attempted_qn|estimator|selector|           duration|run_num|qn_count|qnid|         difference|
+-------+-----------+--------------------+--------------------+---------+--------+-------------------+-------+--------+----+-------------------+
|      0|  -1.070397|  -2.449999990910292|[35.0, 46.0, 69.0...|  ternary|  random|0.17400169372558594|      0|      30|65.0|  1.379602990910292|
|      1|   0.023275|   0.420492659192532|[66.0, 33.0, 18.0...|  ternary|  random| 0.1559460163116455|      0|      30|46.0| -0.397217659192532|
|      2|  -0.881079|  -0.321411498915404|[35.0, 6.0, 19.0,...|  ternary|  random|0.15687823295593262|      0|      30|59.0| -0.559667501084596|
|      3|  -0.291742|  0.6451042841561141|[46.0, 6.0, 5.0, ...|  ternary|  random|0.14149069786071777|      0|      30|52.0| -0.93

In [14]:
# Add sample data to catalog, create temporary table 
# this enable use to write sql query to spark
sample.createOrReplaceTempView('sample')

In [16]:
query_statement = "SELECT * FROM sample LIMIT 5"

In [17]:
# running sql statement into sample
spark.sql(query_statement).show()

+-------+-----------+-------------------+--------------------+---------+--------+-------------------+-------+--------+----+------------------+
|stud_id|actual_abil|       testing_abil|        attempted_qn|estimator|selector|           duration|run_num|qn_count|qnid|        difference|
+-------+-----------+-------------------+--------------------+---------+--------+-------------------+-------+--------+----+------------------+
|      0|  -1.070397| -2.449999990910292|[35.0, 46.0, 69.0...|  ternary|  random|0.17400169372558594|      0|      30|65.0| 1.379602990910292|
|      1|   0.023275|  0.420492659192532|[66.0, 33.0, 18.0...|  ternary|  random| 0.1559460163116455|      0|      30|46.0|-0.397217659192532|
|      2|  -0.881079| -0.321411498915404|[35.0, 6.0, 19.0,...|  ternary|  random|0.15687823295593262|      0|      30|59.0|-0.559667501084596|
|      3|  -0.291742| 0.6451042841561141|[46.0, 6.0, 5.0, ...|  ternary|  random|0.14149069786071777|      0|      30|52.0|-0.936846284156114|

In [18]:
# convert to pandas dataframe
spark.sql(query_statement).toPandas()

,stud_id,actual_abil,testing_abil,attempted_qn,estimator,selector,duration,run_num,qn_count,qnid,difference
0,0,-1.070397,-2.449999990910292,"[35.0, 46.0, 69.0, 53.0, 9.0, 7.0, 15.0, 38.0,...",ternary,random,0.17400169372558594,0,30,65.0,1.379602990910292
1,1,0.023275,0.420492659192532,"[66.0, 33.0, 18.0, 36.0, 47.0, 15.0, 80.0, 81....",ternary,random,0.1559460163116455,0,30,46.0,-0.397217659192532
2,2,-0.881079,-0.321411498915404,"[35.0, 6.0, 19.0, 16.0, 13.0, 33.0, 65.0, 71.0...",ternary,random,0.15687823295593262,0,30,59.0,-0.559667501084596
3,3,-0.291742,0.6451042841561141,"[46.0, 6.0, 5.0, 68.0, 11.0, 51.0, 13.0, 22.0,...",ternary,random,0.14149069786071777,0,30,52.0,-0.936846284156114
4,4,-0.13334,-0.8024054316990081,"[6.0, 77.0, 36.0, 75.0, 21.0, 55.0, 65.0, 78.0...",ternary,random,0.08756160736083984,0,30,53.0,0.6690654316990081


In [21]:
# filtering with spark, it will not perform any tranformation till .show() is called --> lazy evaluation
fil_data = sample.where("stud_id %2 ==0")

In [22]:
# this show only the filtered data
fil_data.show()

+-------+-----------+--------------------+--------------------+---------+--------+-------------------+-------+--------+----+--------------------+
|stud_id|actual_abil|        testing_abil|        attempted_qn|estimator|selector|           duration|run_num|qn_count|qnid|          difference|
+-------+-----------+--------------------+--------------------+---------+--------+-------------------+-------+--------+----+--------------------+
|      0|  -1.070397|  -2.449999990910292|[35.0, 46.0, 69.0...|  ternary|  random|0.17400169372558594|      0|      30|65.0|   1.379602990910292|
|      2|  -0.881079|  -0.321411498915404|[35.0, 6.0, 19.0,...|  ternary|  random|0.15687823295593262|      0|      30|59.0|  -0.559667501084596|
|      4|   -0.13334| -0.8024054316990081|[6.0, 77.0, 36.0,...|  ternary|  random|0.08756160736083984|      0|      30|53.0|  0.6690654316990081|
|      6|  -0.212129|  2.2856964384205636|[42.0, 59.0, 65.0...|  ternary|  random|0.08495688438415527|      0|      30|43.0|